<h1>CS152 Assignment 2: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [1]:
# YOUR CODE HERE (OPTIONAL)
import tabulate
import copy
#Now, define the class PuzzleNode:
class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    """
    def __init__(self,state, depth, parent):
        self.state = state #state as of list of lists
        self.depth = depth #depth level of the nodes
        self.f_val = 0  #evalute function value
        self.parent = parent
        self.pruned = False
    #functions that print out the state of the board in beautiful/human-friendly way
    def __str__(self):
        return tabulate(self.state, tablefmt="fancy_grid") 
    #function thatcompares based on the evaluation function
    def __lt__(self, other):
        return self.f_val < other.f_val
    #function that finds the blank tile    
    def find_blank_tile(self):
        for x in range(len(self.state)):
          for y in range(len(self.state)):
            if self.state[x][y] == 0:
              return x,y
    #function that swap multiple tiles and create its following child.
    def swap_create(self):
      n = len(self.state)
      #get the row,col value of empty tile
      x,y = self.find_blank_tile()
      #create list of lists of possible values if move the empty tile
      new_blank_tile = [[x+1,y],[x-1,y],[x,y+1],[x,y-1]]
      lst_new_state = []
      for tile in new_blank_tile:
          #only swap if specific conditions are met.
          if 0<= tile[0] < n and 0 <= tile[1] < n:
                 copycat_state = copy.deepcopy(self.state)
                 copycat_state[x][y], copycat_state[tile[0]][tile[1]] = copycat_state[tile[0]][tile[1]],copycat_state[x][y]
                 new_state = PuzzleNode(copycat_state, self.depth+1, self)
                 lst_new_state.append(new_state)
      return lst_new_state

In [2]:
state =[[1,2,0],[3,4,6],[5,7,8]]
node = PuzzleNode(state, depth=0,parent=None)
a = node.swap_create()
for i in a:
  print(i.state)


[[1, 2, 6], [3, 4, 0], [5, 7, 8]]
[[1, 0, 2], [3, 4, 6], [5, 7, 8]]


<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [3]:
import numpy as np
#function that returns a state as a list.
def flatten_state(state):
    n = len(state)
    state_np = np.array(state).reshape(n,n)
    flatten_state_np = state_np.flatten()
    lst = flatten_state_np.tolist()
    return lst
print(flatten_state([[1,2,0],[3,4,6],[5,7,8]]))

#function that returns a goal list based on given state
def create_goal(state):
  n = len(state)
  flatten_goal = [i for i in range(n**2)]
  return flatten_goal
print(create_goal([[1,2,0],[3,4,6],[5,7,8]]))

#function that return the tile value ( rows and colums)
def find_tile(state,val):
        for i in range(len(state)):
          for j in range(len(state)):
            if state[i][j] == val:
              return i,j
print(find_tile([[1,2,0],[3,4,6],[5,7,8]],3))

[1, 2, 0, 3, 4, 6, 5, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
(1, 0)


In [5]:
# Add any additional code here (e.g. for the memoization extension)
#function memorize will be later used as decorator.
def memorize(h):
  dict_memory = {}
  def check(state):
        if str(state) not in dict_memory:         
            dict_memory[str(state)] = h(state)
        return dict_memory[str(state)]
  return check

@memorize
# Misplaced tiles heuristic
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """
    goal_state = create_goal(state)
    actual_state = flatten_state(state)
    h = 0
    for i in range(1,len(goal_state)):
      if goal_state[i] != actual_state[i]:
        h+=1
    return h
    
# Manhattan distance heuristic
@memorize  
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    m = len(state)
    
    state = np.array(state).reshape(m, m)
        
    cur_state =  flatten_state(state)
    goal_state = create_goal(state)

    manhattan = [abs(a % m - b % m) + abs(a//m - b//m) for
                      a, b in ((cur_state.index(i), goal_state.index(i)) for
                      i in range(1, len(goal_state)))]
        
    return sum(manhattan)
    

@memorize
def h3(state):
    """
    This function returns a heuristic that dominates the Manhattan distance, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Heuristic distance of the state from its solved configuration
    """
    m = len(state)
    toal_conflicts = 0
    #generate goal as list of lists
    goal = np.array(create_goal(state)).reshape(len(state),len(state)).tolist()
    row_conflicts = 0
    # check each row
    for i in range(m):
        row = state[i]
        conflicts = 0
        # cross-check each element 
        for j in range(len(row)):
            for h in range(len(row)):
                if row[h] != 0 and j != h and row[j] != 0:
                    # coordinates in the state 
                    a1, b1 = find_tile(state, row[j])
                    a2, b2 = find_tile(state, row[h])
                    a3, b3 = find_tile(goal, row[j])
                    a4, b4 = find_tile(goal, row[h])
                    
                    if a1 == a3 and a2 == a4 and a1 == a2 and b1 < b2 and b3 > b4 and row[j] > row[h]:
                        row_conflicts += 1
        if conflicts == len(row):
            row_conflicts += conflicts - 1
        else:
            row_conflicts += conflicts
    mht_distance = h2(state)
    state_t = [list(i) for i in zip(*state)]
    goal_t = [list(i) for i in zip(*goal)]
    col_conflicts = 0
    # check each column
    for i in range(m):
        col = state_t[i]
        conflicts = 0
        # cross-check each element 
        for j in range(len(col)):
            for h in range(len(col)):
                if col[h] != 0 and j != h and col[j] != 0:
                    # coordinates in the state 
                    a1, b1 = find_tile(state, col[j])
                    a2, b2 = find_tile(state, col[h])
                    a3, b3 = find_tile(goal, col[j])
                    a4, b4 = find_tile(goal, col[h])
                    
                    if a1 == a3 and a2 == a4 and a1 == a2 and b1 < b2 and b3 > b4 and col[j] > col[h]:
                        conflicts += 1
        if conflicts == len(col):
            col_conflicts += conflicts - 1 
        else:
            col_conflicts += conflicts
    
    return mht_distance + 2*(row_conflicts + col_conflicts)

# If you implement more than 3 heuristics, then add any extra heuristic functions onto the end of this list.
heuristics = [h1, h2, h3]
# h2(state=[[1,2,0],[3,4,6],[5,7,8]])

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [6]:
def check_format(state):
  m = len(state)
  #check if the state is in the right format (n*n)
  for i in state:
    if len(i) != m:
      return False
  lst_state = flatten_state(state)
  #check if there is duplicates in the list.
  for i in lst_state:
    if lst_state.count(i)>1 == True:
      return False
  return True

def count_inversions(state):
  #flatten the list of lists into a list
  lst_state = flatten_state(state)
  count = 0
  n_titles = (len(state)**2)
  #for loop to see how count inverstions
  for i in range(len(lst_state)-1):
    for j in range(i+1,len(lst_state)):
      if lst_state[i]>lst_state[j] and lst_state[i] and lst_state[j]:
        count +=1
  #return True for even count, false for odd
  return count%2==0

#check if the state is solvable by counting inversions and its len.
def solvable(state):
  if len(state)%2 ==0:
    row,col = find_tile(state,0)
    if row %2 ==0 and count_inversions(state) == False:
      return True
    if row %2 ==1:
      return True
  if len(state)%2 ==1 and count_inversions(state) == True:
    return True
  return False

solvable([[8,1,2],[0,4,3],[7,6,5]])
  

False

In [7]:
state = [[8,1,2],[0,4,3],[7,6,5]]
create_goal(state)
goal = np.array(create_goal(state)).reshape(len(state),len(state)).tolist()
print(goal)

[[0, 1, 2], [3, 4, 5], [6, 7, 8]]


In [8]:
#The following code is heavily influenced by A*search implementation from session 3.1 CS152 taught by Prof.Shekhar

from queue import PriorityQueue

def solvePuzzle(state, heuristic):
     #create a goal state as list of lists
     goal_state = np.array(create_goal(state)).reshape(len(state),len(state)).tolist()
     
     #initial values for output
     max_frontier =0
     steps = 0
     exp = 0
     opt_path = None
     err =0

     #keeps track of the frontier size
     current_frontier_size = 0

     #check_format of the state input
     if check_format(state) is False:
              err = -1
              return steps,exp, max_frontier, opt_path,err

     #check if the state is solvable.         
     if solvable(state) is False:
              err = -2 
              return steps,exp, max_frontier, opt_path,err
     #intialize priority queue
     frontier = PriorityQueue()

     #initialize state as initial node
     initial_node = PuzzleNode(state,0,None)

     #calculate the f_val for the initial node by its depth + heuristic value.
     initial_node.f_val = initial_node.depth +heuristic(initial_node.state)
     
     #establish a dictionary to keep track of the visited node 
     visited = {str(initial_node.state):initial_node}

     frontier.put(initial_node)
     current_frontier_size += 1


     while not frontier.empty():
              # updates max_frontier 
              if max_frontier < current_frontier_size:
                 max_frontier = current_frontier_size
              #get the current node from the queu
              cur_node = frontier.get()
              #reduce the size of frontier
              current_frontier_size -= 1
              #check if the current node is pruned
              if cur_node.pruned:
                       continue
              #check if the current node is the goal -> break          
              if cur_node.state == goal_state:
                       break
              else:
                       #create a list of its following states based on the previous state
                       children = cur_node.swap_create()
                       #increase the expansion by 1
                       exp+=1
                       #for loop through each extended state
                       for child in children:
                                 # if the state is already visited
                                 if str(child.state) in visited.keys():
                                     #check if the depth/level of its visited child's state is larger than the current one 
                                     if visited[str(child.state)].depth > child.depth:
                                                    visited[str(child.state)].pruned = True
                                     else:
                                                    continue
                                 #calculate the f value based on its depth and call heuristic function.
                                 child.f_val = child.depth + heuristic(child.state)
                                 #add child state to a queue
                                 frontier.put(child)
                                 current_frontier_size +=1
                                 visited[str(child.state)] = child
      # Reconstruct the optimal path
     opt_path = [cur_node.state]
      # backtack till there are no more parents
     while cur_node.parent:
          opt_path.append((cur_node.parent).state)
          cur_node = cur_node.parent
     opt_path = opt_path[::-1]
    
     steps = len(opt_path) - 1
    
     return steps, exp, max_frontier, opt_path, err
solvePuzzle([[0,2,1],[3,5,4],[6,7,8]],heuristics[0])

(14,
 277,
 180,
 [[[0, 2, 1], [3, 5, 4], [6, 7, 8]],
  [[3, 2, 1], [0, 5, 4], [6, 7, 8]],
  [[3, 2, 1], [5, 0, 4], [6, 7, 8]],
  [[3, 0, 1], [5, 2, 4], [6, 7, 8]],
  [[3, 1, 0], [5, 2, 4], [6, 7, 8]],
  [[3, 1, 4], [5, 2, 0], [6, 7, 8]],
  [[3, 1, 4], [5, 0, 2], [6, 7, 8]],
  [[3, 1, 4], [0, 5, 2], [6, 7, 8]],
  [[0, 1, 4], [3, 5, 2], [6, 7, 8]],
  [[1, 0, 4], [3, 5, 2], [6, 7, 8]],
  [[1, 4, 0], [3, 5, 2], [6, 7, 8]],
  [[1, 4, 2], [3, 5, 0], [6, 7, 8]],
  [[1, 4, 2], [3, 0, 5], [6, 7, 8]],
  [[1, 0, 2], [3, 4, 5], [6, 7, 8]],
  [[0, 1, 2], [3, 4, 5], [6, 7, 8]]],
 0)

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [ ]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [14]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [15]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


In [16]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [17]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

In [18]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
    assert(dom > 0)

AssertionError: ignored

The current test failed because in the first puzzle, there is no linear conflicts, making manhattan heuristics and linear conflicts heuristic no different. However, this case is special and in other cases, linear conflicts dominates the mahattan one.

In [20]:
dom = 0
for i in range(1,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
    assert(dom > 0)

In [ ]:
## Memoization test - will be carried out after submission

Reference:
1. N-puzzle solver using A* with Manhattan + Linear Conflict, 2021. Retrieved from: https://codereview.stackexchange.com/questions/205667/n-puzzle-solver-using-a-with-manhattan-linear-conflict
2. Looking into k-puzzle Heuristics, Ding Yuchen, 2020. Retrieved from: https://medium.com/swlh/looking-into-k-puzzle-heuristics-6189318eaca2
3. Hasson Mayer Yung, 1992, Linear Conflicts. Retrieved from: https://cse.sc.edu/~mgv/csce580sp15/gradPres/HanssonMayerYung1992.pdf
4. GeeksforGeeks, 2021. Memoization using decorators in Python. Retrieved from: https://www.geeksforgeeks.org/memoization-using-decorators-in-python/
5. Linear Conflict and Mahattan, Youtube, 2021. Retrieved from : https://www.youtube.com/watch?v=8t3lPD2Qtao
